In [1]:
import pandas as pd
from datetime import datetime, timedelta
import os

The below cell will Load CSVs into Pandas and create a MultiIndexed DataFrame
It will also convert Date column to datetime format.

In [2]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'GOOGL']
df_list = []

for ticker in tickers:
    file_path = f"{ticker}.csv"
    df = pd.read_csv(file_path)

    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Add Ticker column if not already present
    if 'Ticker' not in df.columns:
        df['Ticker'] = ticker

    df_list.append(df)

# Concatenate and set MultiIndex
combined_df = pd.concat(df_list)
combined_df.set_index(['Ticker', 'Date'], inplace=True)


The below code will sort data by date within each ticker and handle the missing values.

In [3]:
# Sort index to ensure proper ordering
combined_df.sort_index(inplace=True)

# Interpolate missing values within each Ticker group
combined_df = combined_df.groupby(level=0).apply(lambda group: group.interpolate(method='linear')).droplevel(0)

# Forward fill for any remaining NaNs
combined_df = combined_df.groupby(level=0).apply(lambda group: group.ffill()).droplevel(0)


The below code will filter the data to include only the last 10 years

In [4]:
#ten_years_ago = datetime.today() - timedelta(days=365 * 10)
combined_df = combined_df.reset_index()  # Bring 'Date' back to columns for filtering

# Filter and reapply MultiIndex
#combined_df = combined_df[combined_df['Date'] >= ten_years_ago]
mask = (combined_df['Date'] >= '2007-01-01') & (combined_df['Date'] <= '2017-12-31')
combined_df=combined_df[mask]
combined_df.set_index(['Ticker', 'Date'], inplace=True)


In [5]:
print(combined_df.columns)
print(combined_df.index.names)


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'OpenInt'], dtype='object')
['Ticker', 'Date']


In [6]:
# See cleaned dataset
combined_df.head()


Open    High     Low   Close     Volume  OpenInt
Ticker Date                                                          
AAPL   2007-01-03  11.049  11.087  10.486  10.731  345302870        0
       2007-01-04  10.780  11.007  10.733  10.969  236252357        0
       2007-01-05  10.992  11.038  10.807  10.893  232773093        0
       2007-01-08  11.011  11.081  10.922  10.946  222149027        0
       2007-01-09  11.075  11.907  10.906  11.854  933759387        0

DATA TRANSFORMATION (ADD NEW COLUMNS)

In [7]:
# Step 3: Data Transformation

# Calculate daily return: % change in closing price
combined_df['Daily Return'] = combined_df.groupby(level=0)['Close'].pct_change()

# 7-day Moving Average of closing price
combined_df['7D MA'] = combined_df.groupby(level=0)['Close'].transform(lambda x: x.rolling(window=7).mean())

# 30-day Moving Average of closing price
combined_df['30D MA'] = combined_df.groupby(level=0)['Close'].transform(lambda x: x.rolling(window=30).mean())

# Rolling Volatility (30-day std deviation of returns)
combined_df['30D Volatility'] = combined_df.groupby(level=0)['Daily Return'].transform(lambda x: x.rolling(window=30).std())

# Show the result
combined_df.head(40)


Open    High     Low   Close     Volume  OpenInt  \
Ticker Date                                                             
AAPL   2007-01-03  11.049  11.087  10.486  10.731  345302870        0   
       2007-01-04  10.780  11.007  10.733  10.969  236252357        0   
       2007-01-05  10.992  11.038  10.807  10.893  232773093        0   
       2007-01-08  11.011  11.081  10.922  10.946  222149027        0   
       2007-01-09  11.075  11.907  10.906  11.854  933759387        0   
       2007-01-10  12.135  12.526  11.968  12.420  820730037        0   
       2007-01-11  12.289  12.394  12.178  12.267  401661844        0   
       2007-01-12  12.107  12.174  11.939  12.117  366027068        0   
       2007-01-16  12.257  12.452  12.222  12.433  346840092        0   
       2007-01-17  12.489  12.499  12.143  12.161  453218702        0   
       2007-01-18  11.782  11.796  11.405  11.407  659349162        0   
       2007-01-19  11.348  11.480  11.285  11.331  379744435        0   
       2007-01-22  11.418  11.419  10.968  11.116  402540763        0   
       2007-01-23  10.977  11.207  10.951  10.974  336685388        0   
       2007-01-24  11.024  11.160  11.024  11.103  258728936        0   
       2007-01-25  11.159  11.331  11.018  11.045  252639719        0   
       2007-01-26  11.149  11.189  10.885  10.935  275130377        0   
       2007-01-29  11.045  11.097  10.954  11.006  251405514        0   
       2007-01-30  11.077  11.077  10.917  10.956  160943141        0   
       2007-01-31  10.860  11.014  10.802  10.978  238301361        0   
       2007-02-01  11.031  11.048  10.852  10.852  185223259        0   
       2007-02-02  10.776  10.917  10.717  10.852  173327711        0   
       2007-02-05  10.797  10.915  10.750  10.750  161405362        0   
       2007-02-06  10.814  10.817  10.612  10.777  240961071        0   
       2007-02-07  10.821  11.062  10.700  11.032  297417155        0   
       2007-02-08  10.967  11.080  10.938  11.036  189370669        0   
       2007-02-09  10.998  11.038  10.656  10.664  239880362        0   
       2007-02-12  10.811  10.909  10.709  10.870  201764456        0   
       2007-02-13  10.909  10.923  10.796  10.846  161936755        0   
       2007-02-14  10.837  10.967  10.830  10.924  141666808        0   
       2007-02-15  10.915  10.964  10.856  10.912  101411579        0   
       2007-02-16  10.897  10.938  10.841  10.863  111474825        0   
       2007-02-20  10.830  11.034  10.779  11.000  172235557        0   
       2007-02-21  11.021  11.461  11.008  11.424  322183882        0   
       2007-02-22  11.628  11.630  11.335  11.463  233763003        0   
       2007-02-23  11.433  11.569  11.378  11.407  144427846        0   
       2007-02-26  11.507  11.526  11.220  11.352  171792585        0   
       2007-02-27  11.048  11.150  10.683  10.748  319442902        0   
       2007-02-28  10.828  10.961  10.629  10.834  256409965        0   
       2007-03-01  10.757  11.308  10.724  11.149  394757858        0   

                   Daily Return      7D MA     30D MA  30D Volatility  
Ticker Date                                                            
AAPL   2007-01-03           NaN        NaN        NaN             NaN  
       2007-01-04      0.022179        NaN        NaN             NaN  
       2007-01-05     -0.006929        NaN        NaN             NaN  
       2007-01-08      0.004866        NaN        NaN             NaN  
       2007-01-09      0.082953        NaN        NaN             NaN  
       2007-01-10      0.047748        NaN        NaN             NaN  
       2007-01-11     -0.012319  11.440000        NaN             NaN  
       2007-01-12     -0.012228  11.638000        NaN             NaN  
       2007-01-16      0.026079  11.847143        NaN             NaN  
       2007-01-17     -0.021877  12.028286        NaN             NaN  
       2007-01-18     -0.062001  12.094143        NaN             NaN  
       2007-01-19 

EXPLORATORY ANALYSIS

In [8]:
# Group by Ticker and compute average of Daily Return (excluding NaNs)
combined_df = combined_df.dropna(subset=['Daily Return'])
average_returns = combined_df.groupby('Ticker')['Daily Return'].mean()

# Find the stock with the highest average return
highest_avg_return_ticker = average_returns.idxmax()
highest_avg_return_value = average_returns.max()

print(f" Stock with highest average return: {highest_avg_return_ticker} ({highest_avg_return_value:.4f})")
print(average_returns.sort_values(ascending=False))

 Stock with highest average return: TSLA (0.0021)
Ticker
TSLA     0.002101
AMZN     0.001546
AAPL     0.001224
GOOGL    0.000712
MSFT     0.000591
Name: Daily Return, dtype: float64


In [9]:
# Reset index to access Date easily
df_reset = combined_df.reset_index()

# Create Year-Month column
df_reset['YearMonth'] = df_reset['Date'].dt.to_period('M')

# Compute volatility (std of returns) per Ticker per Month
monthly_volatility = df_reset.groupby(['Ticker', 'YearMonth'])['Daily Return'].std()

# Find the highest volatility entry
most_volatile = monthly_volatility.idxmax()
most_volatile_value = monthly_volatility.max()

print(f" Most volatile month: {most_volatile[0]} in {most_volatile[1]} (Volatility = {most_volatile_value:.4f})")


 Most volatile month: TSLA in 2010-06 (Volatility = 0.2884)
